# Planning and Executing a Query Plan

This example demonstrates how to use the OpenAI Function Call ChatCompletion model to plan and execute a query plan in a question-answering system. By breaking down a complex question into smaller sub-questions with defined dependencies, the system can systematically gather the necessary information to answer the main question.

## Motivation

The goal of this example is to showcase how query planning can be used to handle complex questions, facilitate iterative information gathering, automate workflows, and optimize processes. By leveraging the OpenAI Function Call model, you can design and execute a structured plan to find answers effectively.

**Use Cases:**
- Complex question answering
- Iterative information gathering
- Workflow automation
- Process optimization

With the OpenAI Function Call model, you can customize the planning process and integrate it into your specific application to meet your unique requirements.

## Imports

In [7]:
import enum
from typing import List
from pydantic import Field, BaseModel
import instructor
from openai import OpenAI

In [8]:
from rich import pretty, print
pretty.install()

In [9]:
from dotenv import load_dotenv
load_dotenv("../api_keys.env")

True

## Defining the Structures

Let's define the necessary Pydantic models to represent the query plan and the queries.

In [10]:
class QueryType(str, enum.Enum):
    """Enumeration representing the types of queries that can be asked to a question answer system."""

    SINGLE_QUESTION = "SINGLE"
    MERGE_MULTIPLE_RESPONSES = "MERGE_MULTIPLE_RESPONSES"


class Query(BaseModel):
    """Class representing a single question in a query plan."""

    id: int = Field(..., description="Unique id of the query")
    question: str = Field(
        ...,
        description="Question asked using a question answering system",
    )
    dependencies: List[int] = Field(
        default_factory=list,
        description="List of sub questions that need to be answered before asking this question",
    )
    node_type: QueryType = Field(
        default=QueryType.SINGLE_QUESTION,
        description="Type of question, either a single question or a multi-question merge",
    )


class QueryPlan(BaseModel):
    """Container class representing a tree of questions to ask a question answering system."""

    query_graph: List[Query] = Field(
        ..., description="The query graph representing the plan"
    )

    def _dependencies(self, ids: List[int]) -> List[Query]:
        """Returns the dependencies of a query given their ids."""
        return [q for q in self.query_graph if q.id in ids]

## Client Initialization

In [11]:
# Apply the patch to the OpenAI client
# enables response_model keyword
client = instructor.from_openai(OpenAI())

## Planning a Query Plan

Now, let's demonstrate how to plan and execute a query plan using the defined models and the OpenAI API.

In [12]:
def query_planner(question: str) -> QueryPlan:
    PLANNING_MODEL = "gpt-3.5-turbo"

    messages = [
        {
            "role": "system",
            "content": "You are a world class query planning algorithm capable of breaking apart questions into its dependency queries such that the answers can be used to inform the parent question. Do not answer the questions, simply provide a correct compute graph with good specific questions to ask and relevant dependencies. Before you call the function, think step-by-step to get a better understanding of the problem.",
        },
        {
            "role": "user",
            "content": f"Consider: {question}\nGenerate the correct query plan.",
        },
    ]

    root = client.chat.completions.create(
        model=PLANNING_MODEL,
        temperature=0,
        response_model=QueryPlan,
        messages=messages,
        max_tokens=1000,
    )
    return root

# Example usage
plan = query_planner(
    "What is the difference in populations of Canada and the Jason's home country?"
)
print(plan.model_dump())

{
    'query_graph': [
        {
            'id': 1,
            'question': 'What is the population of Canada?',
            'dependencies': [],
            'node_type': <QueryType.SINGLE_QUESTION: 'SINGLE'>
        },
        {
            'id': 2,
            'question': "What is the population of Jason's home country?",
            'dependencies': [],
            'node_type': <QueryType.SINGLE_QUESTION: 'SINGLE'>
        }
    ]
}

## Note on RAG

While we build the query plan in this example, we do not propose a method to actually answer the question. You can implement your own answer function that perhaps makes a retrieval and calls OpenAI for retrieval augmented generation. That step would also make use of function calls but goes beyond the scope of this example.

## Conclusion

In this example, we demonstrated how to use the OpenAI Function Call ChatCompletion model to plan and execute a query plan using a question-answering system. We defined the necessary structures using Pydantic and created a query planner function.

Feel free to modify the code to fit your specific use case and explore other possibilities of using Function Call models to plan and execute complex workflows.